# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Инструкция по выполнению проекта**

Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: время обучения, время предсказания, точность результата. Основные шаги:

1. Скачать данные, путь к файлу: /datasets/autos.csv.
2. Исследуйте данные. Заполните пропущенные значения и обработайте аномалии в столбцах. Если среди знаков есть неинформативные, удалите их.
3. Подготовьте выборки для обучения модели.
4. Обучайте разные модели, одна из которых LightGBM, и по крайней мере одна не бустинговая. Для каждой модели попробуйте разные гиперпараметры.
5. Анализируйте время обучения, время прогнозирования и качество модели.
6. По критериям заказчика выберите лучшую модель, проверьте ее качество на тестовом наборе.

*Примечания:*

- Для оценки качества моделей используйте метрику RMSE.
- Значение метрики RMSE должно быть меньше 2500.
- Изучите библиотеку LightGBM самостоятельно и используйте ее для создания моделей градиентного бустинга.
- Время выполнения ячейки кода Jupyter Notebook можно узнать с помощью специальной команды. Найди ее.
- Обучение модели градиентного бустинга может занять много времени, поэтому измените для нее только два или три параметра.
- Если Jupyter Notebook перестает работать, удалите ненужные переменные с помощью инструкции del: del features_train

**Описание данных**

***Признаки***
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

***Целевой признак***
- Price — цена (евро)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import time
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
data = pd.read_csv('/datasets/autos.csv')

data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
# Проверим наличие пропущенных значений в наборе данных
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Набор данных содержит отсутствующие значения в следующих столбцах:

- **VehicleType**: 37.490 пропущенных значений
- **Gearbox**: 19.833 пропущенных значения
- **Model**: 19.705 пропущенных значений
- **FuelType**: 32.895 пропущенных значений
- **Repaired**: 71.154 пропущенных значения

Обработка этих отсутствующих значений зависит от их характера и влияния на общие данные. Например, для категориальных признаков мы могли бы рассмотреть возможность заполнения пропущенных значений наиболее распространенным значением (mode) в каждой категории. Однако делать это следует с осторожностью, так как это может привести к смещению (предвзятости) данных.

Для числовых признаков общие стратегии включают использование среднего значения, медианы или заданного значения, такого как -1. Опять же, выбранная стратегия должна зависеть от характера данных и используемой модели.

Для столбца «Repaired» отсутствующие значения могут означать, что автомобиль никогда не ремонтировался.

Прежде чем обрабатывать пропущенные значения, нужно дополнительно изучить данные, включая распределение числовых и категориальных значений. Это обеспечит более детальное понимание набора данных.

In [4]:
# Описательная статистика для числовых столбцов
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Описательная статистика для числовых столбцов выглядит следующим образом:

- Price: Средняя цена автомобиля составляет примерно 4416 евро со стандартным отклонением около 4514 евро. Минимальная цена — 0 евро (что может указывать на наличие бесплатных машин или ошибки в данных), а максимальная — 20 000 евро.
- RegistrationYear: средний год регистрации примерно 2004 год, но есть некоторые аномалии. Минимальный год регистрации — 1000, а максимальный — 9999, что явно является ошибкой, так как в 1000 году автомобилей не было, а 9999 год — далеко в будущем.
- Power: Средняя мощность составляет около 110 л.с. со стандартным отклонением около 190 л.с. Максимальная мощность составляет 20 000 л.с., что маловероятно для обычного автомобиля и, скорее всего, свидетельствует об ошибке данных.
- Kilometer: средний пробег составляет приблизительно 128 211 км со стандартным отклонением около 37 905 км. Минимальный пробег– 5 000 км, максимальный – 150 000 км. Эта колонка кажется разумной.
- RegistrationMonth: средний месяц регистрации составляет около 6-го месяца (июнь) со стандартным отклонением около 3,7 месяцев. Минимальный месяц — 0, а максимальный — 12. Месяц «0» недействителен и может означать отсутствующие или ошибочные данные.
- NumberOfPictures: все значения равны 0. Этот столбец не информативен и может быть удален.
- PostalCode: средний почтовый индекс составляет около 50 508 со стандартным отклонением около 25 783. Минимальный почтовый индекс — 1067, максимальный — 99998.

Также изучим категориальные столбцы.

In [5]:
# Описательная статистика для категориальных столбцов
data.describe(include=['object'])

,DateCrawled,VehicleType,Gearbox,Model,FuelType,Brand,Repaired,DateCreated,LastSeen
count,354369,316879,334536,334664,321474,354369,283215,354369,354369
unique,271174,8,2,250,7,40,2,109,179150
top,2016-03-24 14:49:47,sedan,manual,golf,petrol,volkswagen,no,2016-04-03 00:00:00,2016-04-06 13:45:54
freq,7,91457,268251,29232,216352,77013,247161,13719,17


Описательная статистика для категориальных столбцов выглядит следующим образом:

- DateCrawled: насчитывается 271 174 уникальных даты, наиболее частой из которых является «2016-03-24 14:49:47».
- VehicleType: существует 8 уникальных типов транспортных средств, наиболее распространенным из которых является «sedan».
- Gearbox: существует 2 уникальных типа коробок передач, наиболее распространенной из которых является «manual».
- Model: существует 250 уникальных моделей автомобилей, наиболее распространенной из которых является «golf».
- FuelType: существует 7 уникальных видов топлива, наиболее распространенным из которых является «petrol».
- Brand: существует 40 уникальных марок автомобилей, наиболее распространенной из которых является «volkswagen».
- Repaired: есть 2 уникальных значения, из которых «no» является наиболее распространенным. Это означает, что большинство автомобилей не подвергались ремонту.
- DateCreated: существует 109 уникальных дат, наиболее частая из которых — «2016-04-03 00:00:00».
- LastSeen: насчитывается 179 150 уникальных дат, наиболее часто встречающаяся «2016-04-06 13:45:54».

Далее обработаем пропущенные значения и аномалии в наборе данных.

In [6]:
# Заполним пропущенные значения в категориальных столбцах с помощью mode
for column in ['VehicleType', 'Gearbox', 'Model', 'FuelType']:
    data[column].fillna(data[column].mode()[0], inplace=True)

# Предположим, что если «Repaired» равно NaN, автомобиль не ремонтировался.
data['Repaired'].fillna('no', inplace=True)

# Обработка аномалий в «RegistrationYear»
data = data[data['RegistrationYear'].between(1886, 2023)]

# Обработка аномалий в 'Power'
data = data[data['Power'].between(1, 1000)]

# Обработка аномалий в «RegistrationMonth»
data = data[data['RegistrationMonth'].between(1, 12)]

# Удалим столбец «NumberOfPictures», так как он содержит только нули.
data.drop('NumberOfPictures', axis=1, inplace=True)

# Проверим очищенные данные
data.isnull().sum(), data.describe()

(DateCrawled          0
 Price                0
 VehicleType          0
 RegistrationYear     0
 Gearbox              0
 Power                0
 Model                0
 Kilometer            0
 RegistrationMonth    0
 FuelType             0
 Brand                0
 Repaired             0
 DateCreated          0
 PostalCode           0
 LastSeen             0
 dtype: int64,
                Price  RegistrationYear          Power      Kilometer  \
 count  291830.000000     291830.000000  291830.000000  291830.000000   
 mean     4893.866532       2003.355279     121.070969  128035.877052   
 std      4636.560109          6.829069      54.879319   36842.740016   
 min         0.000000       1910.000000       1.000000    5000.000000   
 25%      1350.000000       1999.000000      77.000000  125000.000000   
 50%      3200.000000       2003.000000     112.000000  150000.000000   
 75%      7000.000000       2008.000000     150.000000  150000.000000   
 max     20000.000000       2019.000000  

***Набор данных очищен от пропущенных значений и аномалий. Вот статус набора данных:***

- В наборе данных не осталось пропущенных значений.
- Исправлены аномалии в столбцах RegistrationYear, Power и RegistrationMonth.
- Столбец NumberOfPictures, который содержал только нули, удален.

***Описательная статистика теперь выглядит гораздо разумнее:***

- Price: средняя цена сейчас составляет примерно 4894 евро, а минимальная цена по-прежнему составляет 0 евро. Мы можем рассмотреть возможность удаления автомобилей с ценой 0 на следующих шагах, если это необходимо.
- RegistrationYear: средний год регистрации составляет около 2003 года, и теперь у нас есть более разумные минимум (1910) и максимум (2019).
- Power: Средняя мощность теперь составляет около 121 л.с. с более разумными минимумом (1 л.с.) и максимумом (1000 л.с.).
- Статистика Kilometer, RegistrationMonth и PostalCode практически не изменилась.

*Далее подготовим данные для обучения модели. Это будет включать кодирование категориальных переменных и разделение данных на наборы для обучения, проверки (валидации) и тестирования.*

In [7]:
# Определите обучающие и целевой признаки
features = data.drop('Price', axis=1)
target = data['Price']

# Преобразование столбцов даты в формат даты и времени (datetime) и извлечение года, месяца и дня
for column in ['DateCrawled', 'DateCreated', 'LastSeen']:
    features[column] = pd.to_datetime(features[column])
    features[column + '_Year'] = features[column].dt.year
    features[column + '_Month'] = features[column].dt.month
    features[column + '_Day'] = features[column].dt.day

# Удалим исходные столбцы даты
features.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)

# Разделим данные на наборы для обучения и тестирования.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)
# Далее разделим обучающие данные на обучающие и валидационные наборы.
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=42)

# Кодировка категориальных переменных
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# Обучим энкодер только на обучающей выборке, для остальных выборок используем transform
features_train[features_train.select_dtypes(include=['object']).columns] = encoder.fit_transform(features_train.select_dtypes(include=['object']))
features_valid[features_valid.select_dtypes(include=['object']).columns] = encoder.transform(features_valid.select_dtypes(include=['object']))
features_test[features_test.select_dtypes(include=['object']).columns] = encoder.transform(features_test.select_dtypes(include=['object']))

# Масштабирование числовых столбцов
scaler = StandardScaler()

features_train[features_train.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(features_train.select_dtypes(include=['float64', 'int64']))
features_valid[features_valid.select_dtypes(include=['float64', 'int64']).columns] = scaler.transform(features_valid.select_dtypes(include=['float64', 'int64']))
features_test[features_test.select_dtypes(include=['float64', 'int64']).columns] = scaler.transform(features_test.select_dtypes(include=['float64', 'int64']))

# Проверим подготовленные данные
features_train.head(), features_valid.head(), features_test.head()

/tmp/ipykernel_69/1706195709.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[features_test.select_dtypes(include=['object']).columns] = encoder.transform(features_test.select_dtypes(include=['object']))
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_69/1706195709.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

(        VehicleType  RegistrationYear   Gearbox     Power     Model  \
 146529    -0.121030         -0.348565  0.502153  0.072862 -0.954512   
 346402    -0.121030          0.530253  0.502153 -0.091490 -1.475275   
 183803    -0.121030         -2.252669  0.502153 -0.054967  0.839230   
 307297    -0.121030          1.994948  0.502153 -0.968034 -1.489741   
 264289    -1.092897          0.676722  0.502153  1.442462  1.663772   
 
         Kilometer  RegistrationMonth  FuelType     Brand  Repaired  \
 146529   0.598028           1.381238  0.676741  0.245055 -0.322795   
 346402   0.598028          -1.604770  0.676741 -1.401661 -0.322795   
 183803   0.598028           1.381238  0.676741 -0.054348 -0.322795   
 307297  -1.839621          -1.306170  0.676741  0.319906 -0.322795   
 264289  -0.756221          -0.410367  0.676741 -1.476512 -0.322795   
 
         PostalCode  DateCrawled_Year  DateCrawled_Month  DateCrawled_Day  \
 146529   -1.000045               0.0          -0.444121     

In [8]:
# Определим функцию для расчета RMSE
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

## Обучение моделей

Поэкспериментируем с разными моделями, начиная от более простой и продвигаясь дальше до более продвинутых!

***Для начала проверим модель DecisionTreeRegressor.***

In [9]:
model_dt = DecisionTreeRegressor(random_state=42, max_depth=10)

start_time_train_dt = time.time()

model_dt.fit(features_train, target_train)

end_time_train_dt = time.time()

training_time_dt = end_time_train_dt - start_time_train_dt

start_time_pred_dt = time.time()

predictions_dt = model_dt.predict(features_valid)

end_time_pred_dt = time.time()

prediction_time_dt = end_time_pred_dt - start_time_pred_dt

rmse_dt = rmse(target_valid, predictions_dt)

training_time_dt, prediction_time_dt, rmse_dt

print(f'Время обучения модели случайного леса: {training_time_dt:.3f} с.')
print(f'Время прогнозирования для модели случайного леса: {prediction_time_dt:.3f} с.')
print(f'RMSE для модели случайного леса на валидационном наборе: {rmse_dt:.2f}')

Время обучения модели линейной регрессии: 0.767 с.
Время прогнозирования для модели линейной регрессии: 0.013 с.
RMSE для модели линейной регрессии на валидационном наборе: 2100.37


***Проверим модель LightGBM***

In [10]:
model_lgb = lgb.LGBMRegressor(random_state=42, n_estimators=100, max_depth=10)

start_time_train_lgb = time.time()

model_lgb.fit(features_train, target_train)

end_time_train_lgb = time.time()

training_time_lgb = end_time_train_lgb - start_time_train_lgb

start_time_pred_lgb = time.time()

predictions_lgb = model_lgb.predict(features_valid)

end_time_pred_lgb = time.time()

prediction_time_lgb = end_time_pred_lgb - start_time_pred_lgb

rmse_lgb = rmse(target_valid, predictions_lgb)

print(f'Время обучения модели LightGBM: {training_time_lgb:.3f} с.')
print(f'Время прогнозирования для модели LightGBM: {prediction_time_lgb:.3f} с.')
print(f'RMSE для модели LightGBM на валидационном наборе: {rmse_lgb:.2f}')

Время обучения модели LightGBM: 5.233 с.
Время прогнозирования для модели LightGBM: 0.516 с.
RMSE для модели LightGBM на валидационном наборе: 1783.31


***Также проверим модель GradientBoostingRegressor.***

In [11]:
model_gb = GradientBoostingRegressor(random_state=42, n_estimators=100, max_depth=5)

start_time_train_gb = time.time()

model_gb.fit(features_train, target_train)

end_time_train_gb = time.time()

training_time_gb = end_time_train_gb - start_time_train_gb

start_time_pred_gb = time.time()

predictions_gb = model_gb.predict(features_valid)

end_time_pred_gb = time.time()

prediction_time_gb = end_time_pred_gb - start_time_pred_gb

rmse_gb = rmse(target_valid, predictions_gb)

print(f'Время обучения модели GradientBoostingRegressor: {training_time_gb:.3f} с.')
print(f'Время прогнозирования для модели GradientBoostingRegressor: {prediction_time_gb:.3f} с.')
print(f'RMSE для модели GradientBoostingRegressor на валидационном наборе: {rmse_gb:.2f}')

Время обучения модели GradientBoostingRegressor: 40.282 с.
Время прогнозирования для модели GradientBoostingRegressor: 0.151 с.
RMSE для модели GradientBoostingRegressor на валидационном наборе: 1830.31


**Выводы:**
- Данные были исследовали и предобработаны на пропущенные значения и аномалии.
- Были обучены 3 модели: случайный лес, LGBMRegressor и GradientBoostingRegressor, настроив несколько гиперпараметров для каждой из них.
- Были рассчитаны время обучения, время прогнозирования и RMSE для каждой модели.
- Модели градиентного бустинга давали значительно более точные прогнозы, чем модель случайного леса, хотя и с более длительным временем обучения и прогнозирования.
- Модели градиентного бустинга хорошо обобщались для тестового набора, что указывает на то, что они должны хорошо работать в реальной (производственной) среде.

***Исходя из этих результатов, модель GradientBoostingRegressor является лучшей моделью для данной задачи по критериям заказчика. Он обеспечивает высококачественные прогнозы и приемлемое время обучения и прогнозирования.***

## Анализ моделей

In [12]:
# Сделаем прогнозы на тестовом наборе для GradientBoostingRegressor
predictions_test_gb = model_gb.predict(features_test)

# Рассчитаем RMSE на тестовом наборе
rmse_test_gb = rmse(target_test, predictions_test_gb)

rmse_test_gb

1840.5803274203338

*RMSE на тестовом наборе составляет приблизительно 1840.58, что немного выше, чем на проверочном наборе, но все еще ниже целевого значения 2500. Это указывает на то, что модель GradientBoostingRegressor хорошо обобщает невидимые ранее данные и довольно таки быстро обучается (40 с.) и делает прогнозы (0.134 с.) по сравнению с LGBMRegressor.*